## Inisialisasi Spark

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Frequent Itemsets") \
    .getOrCreate()

## Import Dataset

In [2]:
df = spark.read.csv("D:/Spark/Tugas 3/data.csv", header=True, inferSchema=True)

In [3]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [4]:
#Pilih kolom untuk backup
backup = df.selectExpr(['InvoiceNo as ID', 'StockCode as Item', 'Description'])

In [5]:
#Pilih kolom untuk di train
df = df.selectExpr(['InvoiceNo as ID', 'StockCode as Item'])

In [6]:
df.show()

+------+------+
|    ID|  Item|
+------+------+
|536365|85123A|
|536365| 71053|
|536365|84406B|
|536365|84029G|
|536365|84029E|
|536365| 22752|
|536365| 21730|
|536366| 22633|
|536366| 22632|
|536367| 84879|
|536367| 22745|
|536367| 22748|
|536367| 22749|
|536367| 22310|
|536367| 84969|
|536367| 22623|
|536367| 22622|
|536367| 21754|
|536367| 21755|
|536367| 21777|
+------+------+
only showing top 20 rows



In [7]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Item: string (nullable = true)



## Grouping dan Menghilangkan Duplikasi

In [8]:
#Grouping
from pyspark.sql import functions

df2 = df.groupby("ID").agg(functions.collect_set("Item").alias('Items'))

In [9]:
df2.show()

+------+--------------------+
|    ID|               Items|
+------+--------------------+
|536596|[22900, 22114, 84...|
|536938|[22112, 21931, 84...|
|537252|             [22197]|
|537691|[22505, 46000R, 2...|
|538041|             [22145]|
|538184|[22561, 22147, 21...|
|538517|[22749, 21212, 22...|
|538879|[21212, 22759, 22...|
|539275|[22083, 22150, 22...|
|539630|[22111, 22971, 22...|
|540499|[22697, 22796, 21...|
|540540|[22111, 22834, 22...|
|540976|[22413, 21212, 22...|
|541432|[22113, 22457, 21...|
|541518|[21212, 22432, 22...|
|541783|[22561, 22697, 22...|
|542026|[22398, 22194, 22...|
|542375|[22629, 21731, 22...|
|543641|[22645, 75131, 22...|
|544303|[84596L, 22931, 8...|
+------+--------------------+
only showing top 20 rows



In [10]:
#Menghilangkan duplikasi

from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf

df_distinct = udf(lambda row: list(set(row)), ArrayType(StringType()))
df2 = df2.withColumn("items_clear", df_distinct("Items"))

In [11]:
df2.show()

+------+--------------------+--------------------+
|    ID|               Items|         items_clear|
+------+--------------------+--------------------+
|536596|[22900, 22114, 84...|[84926A, 21624, 2...|
|536938|[22112, 21931, 84...|[21479, 84997B, 2...|
|537252|             [22197]|             [22197]|
|537691|[22505, 46000R, 2...|[22149, 20975, 22...|
|538041|             [22145]|             [22145]|
|538184|[22561, 22147, 21...|[22492, 22561, 48...|
|538517|[22749, 21212, 22...|[22197, 22844, 22...|
|538879|[21212, 22759, 22...|[22983, 22593, 22...|
|539275|[22083, 22150, 22...|[22423, 21914, 22...|
|539630|[22111, 22971, 22...|[22988, 84347, 22...|
|540499|[22697, 22796, 21...|[21755, 22699, 84...|
|540540|[22111, 22834, 22...|[22555, 22551, 22...|
|540976|[22413, 21212, 22...|[21110, 22207, 84...|
|541432|[22113, 22457, 21...|[22113, 22457, 21...|
|541518|[21212, 22432, 22...|[20724, 21982, 20...|
|541783|[22561, 22697, 22...|[22197, 22961, 84...|
|542026|[22398, 22194, 22...|[2

In [12]:
df2.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- items_clear: array (nullable = true)
 |    |-- element: string (containsNull = true)



## FP Growth

In [13]:
from pyspark.ml.fpm import FPGrowth

In [14]:
fpGrowth = FPGrowth(itemsCol="items_clear", minSupport=0.3, minConfidence=0.75)
model = fpGrowth.fit(df2)

In [15]:
model.freqItemsets.show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [16]:
model.associationRules.show()

+----------+----------+----------+
|antecedent|consequent|confidence|
+----------+----------+----------+
+----------+----------+----------+



Karena hasilnya null maka lakukan fp growth lagi dengan nilai minimun support dan minimum confidence yang berbeda

In [17]:
fpGrowth2 = FPGrowth(itemsCol="items_clear", minSupport=0.05, minConfidence=0.5)
model2 = fpGrowth2.fit(df2)

In [18]:
model2.freqItemsets.show()

+--------+----+
|   items|freq|
+--------+----+
|[85123A]|2246|
| [22423]|2172|
|[85099B]|2135|
| [47566]|1706|
| [20725]|1608|
| [84879]|1468|
| [22720]|1462|
| [22197]|1442|
| [21212]|1334|
| [22383]|1306|
| [20727]|1295|
+--------+----+



In [19]:
model2.associationRules.show()

+----------+----------+----------+
|antecedent|consequent|confidence|
+----------+----------+----------+
+----------+----------+----------+



Walaupun terdapat nilai frekuensinya namun karena hasil confidencenya null maka lakukan fp growth lagi dengan nilai minimun support dan minimum confidence yang lebih rendah lagi

In [20]:
fpGrowth3 = FPGrowth(itemsCol="items_clear", minSupport=0.025, minConfidence=0.25)
model3 = fpGrowth3.fit(df2)

In [21]:
model3.freqItemsets.show()

+---------------+----+
|          items|freq|
+---------------+----+
|       [85123A]|2246|
|        [22423]|2172|
|       [85099B]|2135|
|        [47566]|1706|
|        [20725]|1608|
|        [84879]|1468|
|        [22720]|1462|
|        [22197]|1442|
|        [21212]|1334|
|        [22383]|1306|
| [22383, 20725]| 663|
|        [20727]|1295|
| [20727, 20725]| 648|
|        [22457]|1266|
|         [POST]|1254|
|        [23203]|1249|
|        [22386]|1231|
|[22386, 85099B]| 833|
|        [22960]|1220|
|        [22469]|1214|
+---------------+----+
only showing top 20 rows



In [22]:
model3.associationRules.show()

+----------+----------+-------------------+
|antecedent|consequent|         confidence|
+----------+----------+-------------------+
|   [22699]|   [22697]|                0.7|
|   [22386]|  [85099B]| 0.6766856214459789|
|   [20727]|   [20725]| 0.5003861003861004|
|   [20725]|   [22383]| 0.4123134328358209|
|   [20725]|   [20727]|0.40298507462686567|
|  [85099B]|   [22386]| 0.3901639344262295|
|  [85099B]|   [21931]| 0.3433255269320843|
|  [85099B]|   [22411]| 0.3199063231850117|
|   [22411]|  [85099B]| 0.5754001684919966|
|   [22383]|   [20725]| 0.5076569678407351|
|   [21931]|  [85099B]| 0.6103247293921732|
|   [22697]|   [22699]| 0.7417218543046358|
+----------+----------+-------------------+



In [23]:
dataframe=spark.createDataFrame([
            ('0',['22386'])
        ],['ID','items_clear'])

In [24]:
dataframe.show()

+---+-----------+
| ID|items_clear|
+---+-----------+
|  0|    [22386]|
+---+-----------+



In [25]:
model3.transform(dataframe).show()

+---+-----------+----------+
| ID|items_clear|prediction|
+---+-----------+----------+
|  0|    [22386]|  [85099B]|
+---+-----------+----------+



## Kesimpulan

In [27]:
backup.createOrReplaceTempView("ecommerce")

In [29]:
#cari deskripsi StockCode 22386
query=spark.sql("SELECT DISTINCT Description FROM ecommerce WHERE Item = '22386'")
query.show()

+--------------------+
|         Description|
+--------------------+
|JUMBO BAG PINK PO...|
+--------------------+



In [30]:
#cari deskripsi StockCode 85099B
query2=spark.sql("SELECT DISTINCT Description FROM ecommerce WHERE Item = '85099B'")
query2.show()

+--------------------+
|         Description|
+--------------------+
|JUMBO BAG RED RET...|
+--------------------+



## Kesimpulan

Saat seseorang membeli JUMBO BAG PINK POLKADOT, mereka juga biasanya membeli JUMBO BAG RED RETROSPOT